In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 데이터 로드
import pandas as pd
df_r= pd.read_csv('/content/drive/MyDrive/BDA_안심데이터 공모전/위경도포함 정류장정보/노선_위경도포함_하남.csv',index_col=0)

df_r

,행정동,Unnamed: 0,Longitude,Latitude
0,41450510.0,655,127.223977,37.519233
1,41450530.0,656,127.223977,37.519233
2,41450560.0,657,127.223977,37.519233
3,41450610.0,658,127.223977,37.519233
4,41450620.0,659,127.223977,37.519233
5,41450520.0,661,127.214020,37.541764
6,41450590.0,662,127.214020,37.541764
7,41450600.0,663,127.214020,37.541764
8,41450580.0,666,127.204693,37.552532


In [5]:
# 중복값 제거 후 인덱스 초기화
df_r['행정동'] = df_r['행정동'].drop_duplicates().reset_index(drop=True)

# 인덱스가 동일한 경우, 데이터가 부족할 수 있으므로, 데이터프레임의 길이에 맞추기
df_r = df_r.loc[df_r['행정동'].notnull()]

df_r

,행정동,Unnamed: 0,Longitude,Latitude
0,41450510.0,655,127.223977,37.519233
1,41450530.0,656,127.223977,37.519233
2,41450560.0,657,127.223977,37.519233
3,41450610.0,658,127.223977,37.519233
4,41450620.0,659,127.223977,37.519233
5,41450520.0,661,127.214020,37.541764
6,41450590.0,662,127.214020,37.541764
7,41450600.0,663,127.214020,37.541764
8,41450580.0,666,127.204693,37.552532


#시각화

In [6]:
import folium
import numpy as np

from geopy.distance import geodesic

# 지도 생성
m_SCC = folium.Map(location=[np.mean(df_r['Latitude']), np.mean(df_r['Longitude'])], zoom_start=11)

# 경로를 저장할 리스트
route = [df_r.iloc[0]]  # 시작점 (첫 번째 좌표)

# 남은 경로 데이터프레임
remaining = df_r.iloc[1:].copy()

# 가까운 점을 찾는 방식으로 경로 연결
while not remaining.empty:
    current_point = route[-1]
    current_coords = (current_point['Latitude'], current_point['Longitude'])

    # 가장 가까운 다음 점 찾기
    distances = remaining.apply(
        lambda row: geodesic(current_coords, (row['Latitude'], row['Longitude'])).meters, axis=1
    )
    next_idx = distances.idxmin()  # 가장 가까운 점의 인덱스
    next_point = remaining.loc[next_idx]

    # 경로에 추가
    route.append(next_point)

    # 남은 점에서 제거
    remaining = remaining.drop(next_idx)

# 최종 경로를 연결
route_df = pd.DataFrame(route)

# 폴리라인 그리기
locations = route_df[['Latitude', 'Longitude']].values.tolist()
folium.PolyLine(locations=locations, color='blue', weight=2.5, opacity=1).add_to(m_SCC)

# 노드 마커 추가
for _, row in route_df.iterrows():
    coords = (row['Latitude'], row['Longitude'])
    folium.Marker(
        location=coords,
        popup=row['행정동'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m_SCC)

# 결과 확인
m_SCC



In [7]:
import pandas as pd

# 순서대로 정렬된 DataFrame을 가져오기 위해 인덱스를 설정
route_df = pd.DataFrame(route)
route_df.reset_index(drop=True, inplace=True)

# 기존 DataFrame의 '행정동' 또는 유사한 고유 식별자를 사용하여 정렬
df_r_sorted = df_r.set_index('행정동').loc[route_df['행정동']].reset_index()

# 결과 확인
df_r_sorted


,행정동,Unnamed: 0,Longitude,Latitude
0,41450510.0,655,127.223977,37.519233
1,41450530.0,656,127.223977,37.519233
2,41450560.0,657,127.223977,37.519233
3,41450610.0,658,127.223977,37.519233
4,41450620.0,659,127.223977,37.519233
5,41450520.0,661,127.214020,37.541764
6,41450590.0,662,127.214020,37.541764
7,41450600.0,663,127.214020,37.541764
8,41450580.0,666,127.204693,37.552532


In [9]:
df_r_sorted.to_csv("/content/drive/MyDrive/BDA_안심데이터 공모전/위경도포함 정류장정보/정류장순서반영_하남.csv")

In [10]:
!pip install osmnx

In [ ]:
import folium
import numpy as np
import osmnx as ox

# OSMnx 그래프 로드
G = ox.graph_from_place('하남시', network_type='all')

# SCC 좌표를 OSMnx 노드에 매핑하기 위한 딕셔너리 생성
scc_coords = {row['행정동']: (row['Latitude'], row['Longitude']) for index, row in df_r.iterrows()}

# OSMnx 노드에 대한 좌표를 가져오기
osmnx_node_coords = {node: (data['y'], data['x']) for node, data in G.nodes(data=True)}

# 최단 경로 계산을 위한 SCC 노드의 OSMnx 노드 찾기
route_nodes = []
for i in range(len(df_r_sorted) - 1):
    start_node = df_r_sorted.iloc[i]['행정동']
    end_node = df_r_sorted.iloc[i + 1]['행정동']

    start_coords = scc_coords[start_node]
    end_coords = scc_coords[end_node]

    # OSMnx에서 노드 검색
    start_osmnx_node = ox.distance.nearest_nodes(G, start_coords[1], start_coords[0])
    end_osmnx_node = ox.distance.nearest_nodes(G, end_coords[1], end_coords[0])

    # 최단 경로 계산
    shortest_path = ox.shortest_path(G, start_osmnx_node, end_osmnx_node, weight='length')
    route_nodes.extend(shortest_path)

# 중복된 노드 제거
route_nodes = list(dict.fromkeys(route_nodes))

# Folium을 사용한 경로 시각화
m_OS = folium.Map(location=[np.mean(df_r_sorted['Latitude']), np.mean(df_r_sorted['Longitude'])], zoom_start=11)

for i in range(len(route_nodes) - 1):
    start_node = route_nodes[i]
    end_node = route_nodes[i + 1]

    start_coords = osmnx_node_coords[start_node]
    end_coords = osmnx_node_coords[end_node]

    # Folium PolyLine에 사용되는 좌표가 (위도, 경도)인지 확인
    folium.PolyLine(
        locations=[(start_coords[0], start_coords[1]), (end_coords[0], end_coords[1])],
        color='blue'
    ).add_to(m_OS)

# 노드 마커 추가
for _, row in df_r.iterrows():
    coords = (row['Latitude'], row['Longitude'])
    folium.Marker(
        location=coords,
        popup=row['행정동'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m_OS)

# 결과 지도 출력
m_OS


In [ ]:
m_SCC.save('/content/drive/MyDrive/BDA_안심데이터 공모전/map/map_하남_SCC.html')
m_OS.save('/content/drive/MyDrive/BDA_안심데이터 공모전/map/map_하남_OSMnx.html')